In [1]:
# !pip install -qU llama-index llama-parse python-dotenv

In [2]:
# !pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

In [1]:
# !pip show torch
# Name: llama-index
# Version: 0.10.35

# Name: llama-index-postprocessor-flag-embedding-reranker
# Version: 0.1.2

Reference notebook: [LlamaParse - A Tool for Parsing Complex Documents.ipynb](https://colab.research.google.com/drive/1IVQkSGwS5kdTiKBwz85PO6vg_WaNx15c?usp=sharing#scrollTo=pLlEWwpFqFrp)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# wd_path = "/content/drive/MyDrive/Colab Notebooks/rag-notebooks"
wd_path = ".."

In [4]:
import sys
import openai
import os
import getpass
import logging

from dotenv import load_dotenv, find_dotenv

In [5]:
# os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [6]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [7]:
# os.environ["HF_TOKEN"] = getpass.getpass("Hugging Face API Key:")

In [8]:
import nest_asyncio

nest_asyncio.apply()

In [9]:
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [10]:
from llama_index.core.callbacks.global_handlers import set_global_handler

set_global_handler("simple")
# set_global_handler("langfuse")

In [11]:
# import llama_parse
# import llama_index
# import os

# llama_parse_path = os.path.dirname(llama_parse.__file__)
# llama_index_path = os.path.dirname(llama_index.__file__)

# print(llama_parse_path)
# print(llama_index_path)

# Helpers

In [38]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.core.node_parser.text.sentence import SentenceSplitter
# assume documents are defined -> extract nodes
from llama_index.core.ingestion import IngestionPipeline

In [56]:
PT_QUESTION_GEN_TMPL = """\
Aqui está o contexto:
{context_str}

Dada a informação contextual, gere {num_questions} perguntas que este contexto pode fornecer respostas específicas \
que dificilmente seriam encontradas em outro lugar.

Resumos de nível mais alto do contexto circundante também podem ser fornecidos. Tente usar esses resumos para gerar melhores \
perguntas que este contexto possa responder.

"""

PT_SUMMARY_EXTRACT_TEMPLATE = """\
Aqui está o conteúdo da seção:
{context_str}

Resuma os principais tópicos e entidades da seção. \

Resumo:"""

In [57]:

def get_metadata_node_parser_pipeline():
    qa_extractor = QuestionsAnsweredExtractor(questions=3)
    
    extractors = [
        # TitleExtractor(nodes=5),
        QuestionsAnsweredExtractor(questions=3, prompt_template=PT_QUESTION_GEN_TMPL),
        SummaryExtractor(summaries=["prev", "self"], prompt_template=PT_SUMMARY_EXTRACT_TEMPLATE),
        # KeywordExtractor(keywords=10),
        # CustomExtractor()
    ]
    
    NODE_PARSER_CHUNK_SIZE = 512
    NODE_PARSER_CHUNK_OVERLAP = 10
    
    
    sentence_splitter = SentenceSplitter.from_defaults(
        chunk_size=NODE_PARSER_CHUNK_SIZE,
        chunk_overlap=NODE_PARSER_CHUNK_OVERLAP
    )
    
    transformations = [sentence_splitter] + extractors
    
    pipeline = IngestionPipeline(
        transformations=transformations
    )
    return pipeline

def print_node_parsing_result(node, metadata_mode, only_metadata):
    print("\n\n----------------------- " + node.id_ + " -----------------------")
    if only_metadata:
        print(node.get_metadata_str(mode=metadata_mode))
    else:
        print(node.get_content(metadata_mode=metadata_mode))

def print_nodes_parsing_result(nodes, metadata_mode="all", only_metadata=False, ids_list=None):
    print("length: ", len(nodes))
    for node in nodes:
        if ids_list is not None:
            if node.id_ in ids_list:
                print_node_parsing_result(node, metadata_mode, only_metadata)
        else:
            print_node_parsing_result(node, metadata_mode, only_metadata)

# Law questions

In [58]:
duvidas = {
    "Devedor": {
        "Contrato de Financiamento": "Quem é o Devedor? (BENEFICIÁRIA / DEVEDOR)",
        "Cédula de Crédito Bancário": "Quem é o  Devedor? (Emitente)"
    },
    "Credor": {
        "Contrato de Financiamento": "Quem é o Credor? (BNDES / FINEP)",
        "Cédula de Crédito Bancário": "Quem é o  Credor? (Credor ou Nome do banco)"
    },
    "Data de Assinatura": {
        "Contrato de Financiamento": "Qual é a data de assinatura? (Costuma estar no final do documento, após a frase `E, por estarem justos e contratados, firmam o presente em 3 (três) vias...`)",
        "Cédula de Crédito Bancário": "Qual é a data de assinatura? (Data de emissão)"
    },
    "Vencimento/Prazo": {
        "Contrato de Financiamento": "Qual é o Vencimento/Prazo? ('liquidar com a última prestação')",
        "Cédula de Crédito Bancário": "Qual é o Vencimento/Prazo? (ou Forma de pagamento, assim costuma aparecer o numero de parcelas)"
    },
    "Valor": {
        "Contrato de Financiamento": "Qual é o Valor (crédito no valor de [...])",
        "Cédula de Crédito Bancário": "Qual é o Valor? (valor do empréstimo)"
    },
    "Objeto": {
        "Contrato de Financiamento": "Qual é a FINALIDADE DO CONTRATO?",
        "Cédula de Crédito Bancário": "Qual é a Destinação do Crédito? (O que será creditado na conta corrente da Emitente?)"
    },
    "Alteração de Controle/ Reorganização": {"Contrato de Financiamento": "O que consta na clausula de Alteração de Controle/ Reorganização? (palavras chaves: 'controle direto e indireto'/ 'Vencimento Antecipado do contrato')",
                         "Cédula de Crédito Bancário": "O que consta na clausula de Alteração de Controle/ Reorganização? (palavras chaves: 'vencimento antecipado', 'inadimplemento' ou 'antecipadamente vencidas' / 'transferido o controle do capital' e/ou 'substituído')"
    },
    "Cessão": {"Contrato de Financiamento": "O que consta na clausula de cessão? (palavras chaves: 'ceder' / 'transferir')",
               "Cédula de Crédito Bancário": "O que consta na clausula de cessão? (palavras chaves: 'ceder', 'transferir', ou 'onerar a qualquer título...')"
    }
}

current_type = "Cédula de Crédito Bancário"
for search, question in duvidas.items():
    print(question)

{'Contrato de Financiamento': 'Quem é o Devedor? (BENEFICIÁRIA / DEVEDOR)', 'Cédula de Crédito Bancário': 'Quem é o  Devedor? (Emitente)'}
{'Contrato de Financiamento': 'Quem é o Credor? (BNDES / FINEP)', 'Cédula de Crédito Bancário': 'Quem é o  Credor? (Credor ou Nome do banco)'}
{'Contrato de Financiamento': 'Qual é a data de assinatura? (Costuma estar no final do documento, após a frase `E, por estarem justos e contratados, firmam o presente em 3 (três) vias...`)', 'Cédula de Crédito Bancário': 'Qual é a data de assinatura? (Data de emissão)'}
{'Contrato de Financiamento': "Qual é o Vencimento/Prazo? ('liquidar com a última prestação')", 'Cédula de Crédito Bancário': 'Qual é o Vencimento/Prazo? (ou Forma de pagamento, assim costuma aparecer o numero de parcelas)'}
{'Contrato de Financiamento': 'Qual é o Valor (crédito no valor de [...])', 'Cédula de Crédito Bancário': 'Qual é o Valor? (valor do empréstimo)'}
{'Contrato de Financiamento': 'Qual é a FINALIDADE DO CONTRATO?', 'Cédula d

In [59]:
pdf_file_path = wd_path + "/data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf"

# Load and Parse PDFs

In [60]:
from llama_parse import LlamaParse  # pip install llama-parse

# parser = LlamaParse(
#     # api_key="...",  # can also be set in your env as LLAMA_CLOUD_API_KEY
#     result_type="markdown"  # "markdown" and "text" are available
# )

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="pt",
    num_workers=2,
)

# sync
# documents = parser.load_data("./my_file.pdf")

# async
# docs0 = await parser.aload_data("../data/1.2.5.17.10.1.1 CCB 511300712 - BB.pdf")

# LlamaIndex Recursive Query Engine

## Extract documents

In [61]:
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core.indices.loading import load_index_from_storage
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
Settings.callback_manager = CallbackManager([llama_debug])

In [62]:
# "DISPOSIÇÕES APLICÁVEIS AOS CONTRATOS DO BNDES".lower()

In [63]:
PERSIST_DIR = wd_path + "/data/storage"

os.path.exists(PERSIST_DIR)

True

In [64]:
# check if storage already exists

get_from_storage = os.path.exists(PERSIST_DIR)
# get_from_storage = False

# node_parser_type = "markdown"
node_parser_type = "metadata"
if not get_from_storage:
    # load the documents and create the index
    # documents = SimpleDirectoryReader(llamaindex_dir + "/data").load_data()
    documents = await parser.aload_data(pdf_file_path)
    if node_parser_type == "markdown":
        node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)
        nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])
        base_nodes, objects = node_parser.get_nodes_and_objects(nodes)
        nodes_to_index = base_nodes+objects
    if node_parser_type == "metadata":
        pipeline = get_metadata_node_parser_pipeline()
        nodes_to_index = pipeline.run(
            documents=documents,
            # nodes=base_nodes + objects,
            in_place=True,
            show_progress=True,
        )
    recursive_index = VectorStoreIndex(nodes=nodes_to_index)
    # index = VectorStoreIndex.from_documents(documents)
    # store it for later
    # saving the embeddings to disk
    # By default, this will save the data to the directory storage, but you can change that by passing a persist_dir parameter.
    recursive_index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    recursive_index = load_index_from_storage(storage_context)

Started parsing the file under job_id e208f21b-c8a8-471c-8778-d8539edf53e3


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                                                                | 0/28 [00:00<?, ?it/s]

** Messages: **
user: Aqui está o contexto:
16 do Decreto nº 6.514, de 22.07.2008, ou outra norma legal que venha substituí-lo, OBRIGANDO-ME(NOS) a informar ao Banco do Brasil S.A., impreterivelmente até a data da liberação de qualquer crédito por mim pleiteado, eventuais fatos ou

- Continua na página 10 -
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 10 -

emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

circunstâncias que possam ensejar o enquadramento nas disposições legais aqui mencionadas.

SISTEMA DE INFORMAÇÕES DE CRÉDITO DO BANCO CENTRAL - SCR -

Declaro-me (amo-nos) cienteí(s) que fui (fomos) comunicado(s) que:

1. os débitos e responsabilidades decorrentes de operações com características de crédito por mim(ínós) realizadas serão registrados no Sistema de Informações de Crédito do Banco Central - SCR;
2. o SCR tem por finalidades f

  4%|███████▋                                                                                                                                                                                                                | 1/28 [00:03<01:28,  3.29s/it]

...

** Messages: **
user: Aqui está o conteúdo da seção:
[Excerpt from document]
questions_this_excerpt_can_answer: 1) Quais são as medidas que a empresa se compromete a adotar para evitar danos ao meio ambiente e cumprir a legislação ambiental aplicável?
2) Como a empresa garante que não está envolvida em práticas como trabalho análogo ao escravo, tráfico de pessoas, exploração de mão de obra infantil e exploração sexual de menores?
3) Como a empresa garante que seus fornecedores diretos também estão comprometidos em proteger o meio ambiente e cumprir a legislação social e trabalhista?

Resumo de nível mais alto: O contexto aborda a responsabilidade socioambiental da empresa, incluindo o cumprimento da legislação ambiental, social e trabalhista, a adoção de medidas para evitar danos ao meio ambiente e a garantia de que não está envolvida em práticas prejudiciais. Também destaca a importância de os fornecedores diretos também se comprometerem com essas práticas responsáveis.
Excerpt:
-----

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎       | 27/28 [00:21<00:00,  1.30it/s]

** Messages: **
user: Aqui está o conteúdo da seção:
[Excerpt from document]
questions_this_excerpt_can_answer: 1. Quais são as obrigações legais que o Decreto nº 6.514 impõe ao informar ao Banco do Brasil S.A. antes da liberação de qualquer crédito pleiteado?
2. Qual é o valor e a data de vencimento da Cédula de Crédito Bancário emitida pela ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA em favor do Banco do Brasil S.A.?
3. Como o Sistema de Informações de Crédito do Banco Central (SCR) funciona e quais são as finalidades do registro de débitos e responsabilidades decorrentes de operações de crédito?
Excerpt:
-----
16 do Decreto nº 6.514, de 22.07.2008, ou outra norma legal que venha substituí-lo, OBRIGANDO-ME(NOS) a informar ao Banco do Brasil S.A., impreterivelmente até a data da liberação de qualquer crédito por mim pleiteado, eventuais fatos ou

- Continua na página 10 -
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 10 -

emitida nesta data por ALLIAGE S/A 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:21<00:00,  1.28it/s]


**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 1.470975 seconds
**********


In [91]:
# from llama_index.core.schema import MetadataMode


# n_index = 0
# nodes_to_index[n_index].excluded_embed_metadata_keys = ["questions_this_excerpt_can_answer", "prev_section_summary", "section_summary"]
# nodes_to_index[n_index]

# print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.EMBED))

In [93]:
# UPDATE DOCUMENTS WITHOUT HAVING TO PARSE AGAIN
for n in nodes_to_index:
    n.excluded_embed_metadata_keys = ["questions_this_excerpt_can_answer", "prev_section_summary", "section_summary"]

- Brasil, Bolsa e Balcão (B3); e por data-base, em cada mês, o dia definido para o débito dos encargos financeiros - indicado no item "DADOS DA OPERAÇÃO

- Continua na página 3 - Ex
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 3 -

Emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

DE CRÉDITO" da introdução. Caso a data-base escolhida seja o dia 29, 30 ou 31, nos meses em que não existirem tais dias, será considerado, como data-base, o primeiro dia do mês subsequente.

FORMA DE PAGAMENTO - Sem prejuízo do vencimento retroestipulado e das exigibilidades previstas nas demais cláusulas, inclusive encargos financeiros, obrigo-me(amos) a pagar ao BANCO DO BRASIL S.A. à dívida resultante desta CEDULA DE CREDITO BANCARIO, conforme parcelas, vencimentos e valores pactuados em DADOS DA OPERAÇÃO DEma CRÉDITO, obrigando-me(nos) ainda a liquidar com a últ

In [94]:
recursive_index = VectorStoreIndex(nodes=nodes_to_index)
# recursive_index.storage_context.persist(persist_dir=PERSIST_DIR)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 1.250255 seconds
**********


## Check metadata and content

In [104]:
n_index = 5
print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.EMBED))
# print(nodes_to_index[n_index].get_content(metadata_mode=MetadataMode.LLM))

- Brasil, Bolsa e Balcão (B3); e por data-base, em cada mês, o dia definido para o débito dos encargos financeiros - indicado no item "DADOS DA OPERAÇÃO

- Continua na página 3 - Ex
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 3 -

Emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

DE CRÉDITO" da introdução. Caso a data-base escolhida seja o dia 29, 30 ou 31, nos meses em que não existirem tais dias, será considerado, como data-base, o primeiro dia do mês subsequente.

FORMA DE PAGAMENTO - Sem prejuízo do vencimento retroestipulado e das exigibilidades previstas nas demais cláusulas, inclusive encargos financeiros, obrigo-me(amos) a pagar ao BANCO DO BRASIL S.A. à dívida resultante desta CEDULA DE CREDITO BANCARIO, conforme parcelas, vencimentos e valores pactuados em DADOS DA OPERAÇÃO DEma CRÉDITO, obrigando-me(nos) ainda a liquidar com a últ

In [24]:
# print(objects[1].metadata["col_schema"])
# print(base_nodes[1].metadata)
# nodes_with_metadata[0].metadata
# TODO: add metadata to these base nodes

In [44]:
def print_node_parsing_result(node, metadata_mode, only_metadata):
    print("\n\n----------------------- " + node.id_ + " -----------------------")
    if only_metadata:
        print(node.get_metadata_str(mode=metadata_mode))
    else:
        print(node.get_content(metadata_mode=metadata_mode))

def print_nodes_parsing_result(nodes, metadata_mode="all", only_metadata=False, ids_list=None):
    print("length: ", len(nodes))
    for node in nodes:
        if ids_list is not None:
            # print(node.id_)
            if node.id_ in ids_list:
                print_node_parsing_result(node, metadata_mode, only_metadata)
        else:
            print_node_parsing_result(node, metadata_mode, only_metadata)

In [46]:
duvidas["Data de Assinatura"][current_type]

'Qual é a data de assinatura? (Data de emissão)'

In [111]:
# nodes_with_metadata[0].metadata["questions_this_excerpt_can_answer"]
# nodes_to_index[0].metadata

for node in nodes_to_index:
    if "ribeirao" in node.get_content().lower():
        print("\n\n-------------------")
        print(node.get_content())
        print(node.id_)



-------------------
neste ato representada por LUIZ ROBERTO 04.78 7T6G7T7/0001-72, neste ato representada por LUIZ ROBERTO KAYSEL CRUZ, Brasileiro(a), ENGENHEIRO, casado(a) sob o regime de comunhão universal de bens, residente e domiciliado em RUA PROFESSOR MARIO ROXO 25, CEP: 14.025-460, município-UF: RIBEIRAO PRETO-SP, portador do(a) CARTEIRA DE IDENTIDADE nr. 57753568 SSP SP e inscrito no CPE/MF sob o nr. 000.277.788-60 e EDUARDO BIAGI, Brasileiro(a), CASADO(A)-COMUNHÃO UNIVERSAL, EMPRESÁRIO, residente em RIBEIRÃO PRETO - SP, na COND COLINA VERDE, 55 KM 312 CS.55, Bairro: COLINA VERDE, CEP: 14,022-050, portador (a) do (a) CARTEIRA DE IDENTIDADE nr. 3.542.871 SSPSP, inscrito(a) no CPF sob o nr. 551.123.078-20 e PEDRA AGROINDUSTRIAL S/A, sediado(a) em
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 2 -

Emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15

In [112]:

# print_nodes_parsing_result(base_nodes)
# print_nodes_parsing_result(objects, "embed")
# print_nodes_parsing_result(objects, "llm")
# print_nodes_parsing_result(nodes_with_metadata, "embed")

# ids_list = ["b7d05e4d-21d4-44a5-90a8-a1a188b20072", "16966cdb-8c94-40ae-8ab7-4719558a0045"]
# ids_list = ["956167cb-3426-4b92-a71c-5a2e725eb167", "ea2982c0-4153-448c-9923-a0bd4181c06a"]
ids_list = ["ea2927aa-13e7-429f-ab2e-62441f32e654"]
print_nodes_parsing_result(nodes_to_index, metadata_mode="all", only_metadata=True, ids_list=ids_list)
# SEEMS LIKE THE IDEA OF RECURSIVE RETRIEVER IS:
# - EMBEDDING RETRIEVAL: WITHOUT THE TABLE NODES CONTENT
# - LLM INTERPRETATION: WITH THE TABLE NODES CONTENT

length:  28


----------------------- ea2927aa-13e7-429f-ab2e-62441f32e654 -----------------------
questions_this_excerpt_can_answer: 1. Qual é o valor da Cédula de Crédito Bancário emitida pela ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA em favor do Banco do Brasil S.A.?
2. Qual é a data de vencimento final da Cédula de Crédito Bancário?
3. Onde devem ser cumpridas as obrigações assumidas nesta Cédula de Crédito Bancário?

Resumo de nível mais alto: Este contexto fornece informações sobre uma Cédula de Crédito Bancário emitida pela ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025. O documento foi aditado, retificado e ratificado, e as obrigações devem ser cumpridas na Agência do Banco do Brasil S.A. em Ribeirão Preto-SP.
prev_section_summary: - Como solicitar correções ou exclusões de informações no SCR
- Números de telefone para entrar em contato com o Banco para sugestões, reclamações ou escla

In [19]:
# print(documents[0].text[:500])

# ## CEDULA DE CREDITO BANCARIO

# INTRODUÇÃO: Nr. 511.300.712

# 1. EMITENTE:

# |Razão ou denominação social:|ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA|
# |---|---|
# |CNPJ:|55.979.736/0001-45|
# |Conta Corrente:|000.003.203-4|
# |Endereço:|ROD ABRAÃO ASSED S/N KM 53 MAIS 450 METROS, RECREIO ANHANGUERA|
# |Cidade:|RIBEIRÃO PRETO-SP|
# |CEP:|14.097-500|
# |E-MAIL:|fabiola.furtadoCalliage-global.com|

# 2. DADOS DA OPERAÇÃO DE CRÉDITO:

# |Valor:|R$25.000.000,00 (vinte e cinco milhões de reais)|
# |---|---|
# |Vencimento:|1

In [ ]:
# from IPython.display import Markdown, display

# # define prompt viewing function
# def display_prompt_dict(prompts_dict):
#     for k, p in prompts_dict.items():
#         text_md = f"**Prompt Key**: {k}" f"**Text:** "
#         display(Markdown(text_md))
#         print(p.get_template())
#         display(Markdown(""))
# prompts_dict = query_engine_base.get_prompts()

# display_prompt_dict(prompts_dict)

## Retriever

- [Node Postprocessor](https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/)

In [150]:
from duvidas import duvidas

duvida = duvidas["Data de Assinatura"][current_type]

'Em que data o documento foi assinado?'

In [ ]:
debug_post_process = False
if debug_post_process:
    from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

    base_retriever = recursive_index.as_retriever(similarity_top_k=15)
    reranker = FlagEmbeddingReranker(
        top_n=5,
        model="BAAI/bge-reranker-large",
    )
    retrievals = processor.postprocess_nodes(base_retriever.retrieve(duvida))
else:
    base_retriever = recursive_index.as_retriever(similarity_top_k=2)
    retrievals = base_retriever.retrieve(duvida)

In [153]:
for n in retrievals:
    display_source_node(n, source_length=5000)

**Node ID:** ea2927aa-13e7-429f-ab2e-62441f32e654<br>**Similarity:** 0.46324079245545213<br>**Text:** Esta CEDULA DE CREDITO BANCARIO poderá ser

Continua na página 11 -

Projeto São Sebastião - arnaldo.bernardi@lefosse.com - Arnaldo Bernardi - Lefosse Advogados - 04/03/2023 18:03:11 - 189.69.21.238
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 11 -

Emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

Aditada, retificada e ratificada mediante termo de aditamento escrito, com os requisitos previstos no "caput", quanto a quantidade de vias e a via negociável, que passará a integrar este Instrumento para todos os fins de direito.

E por ser de minha(nossa) livre e espontânea vontade, assino(amos) esta CEDULA DE CREDITO BANCARIO em 2 (duas) vias de igual teor, para todos os fins de direito.

LOCAL DO PAGAMENTO - Cumprirei(emos) as obrigações assumidas nesta CEDULA DE CREDITO BANCARIO junto à Agência do BANCO DO BRASIL S.A., em que for mantida minha(nossa) conta corrente, for praça de pagamento que fica designada como foro deste Instrumento.

RIBEIRAO PRETO-SP, 18 de novembro de 2020.<br>

**Node ID:** 9a9edca6-04d2-4233-bace-bfec50e61e0e<br>**Similarity:** 0.46311000544492925<br>**Text:** neste ato representada por LUIZ ROBERTO 04.78 7T6G7T7/0001-72, neste ato representada por LUIZ ROBERTO KAYSEL CRUZ, Brasileiro(a), ENGENHEIRO, casado(a) sob o regime de comunhão universal de bens, residente e domiciliado em RUA PROFESSOR MARIO ROXO 25, CEP: 14.025-460, município-UF: RIBEIRAO PRETO-SP, portador do(a) CARTEIRA DE IDENTIDADE nr. 57753568 SSP SP e inscrito no CPE/MF sob o nr. 000.277.788-60 e EDUARDO BIAGI, Brasileiro(a), CASADO(A)-COMUNHÃO UNIVERSAL, EMPRESÁRIO, residente em RIBEIRÃO PRETO - SP, na COND COLINA VERDE, 55 KM 312 CS.55, Bairro: COLINA VERDE, CEP: 14,022-050, portador (a) do (a) CARTEIRA DE IDENTIDADE nr. 3.542.871 SSPSP, inscrito(a) no CPF sob o nr. 551.123.078-20 e PEDRA AGROINDUSTRIAL S/A, sediado(a) em
---
# Continuação da CEDULA DE CREDITO BANCARIO nr. 511.300.712, - Página 2 -

Emitida nesta data por ALLIAGE S/A INDUSTRIAS MEDICO ODONTOLOGICA, em favor do Banco do Brasil S.A., no valor de R$25.000.000,00, com vencimento final em 15/08/2025.

USINA DA PEDRA SN, ZONA RURAL, SERRANA - SP, Cep: 14.150-000 e inscrito(a) no CNPJ sob o nr. 71.304.687/0001-05, E-mail: manfredi€Guspedra.com.br, neste ato representada por LUIZ ROBERTO KAYSEL CRUZ, acima qualificado.<br>

## Query engine

### Base

In [154]:
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, verbose=True)

In [155]:
%%time

from duvidas import duvidas

recursive_search = False
# recursive_search = True
not_to_search = [
    # DONE
    "Objeto", "Data de Assinatura", "Alteração de Controle/ Reorganização", "Cessão", 
     # TODO LATER
     "Observações"]

# to_search = ["Garantias"]
# to_search = ["Cross default"]
# to_search = ["Objeto", "Cessão"]
# to_search = ["Alteração de Controle/ Reorganização"]
to_search = ["Data de Assinatura"]
# to_search = ["Devedor"]
# current_type = "Contrato de Financiamento"
current_type = "Cédula de Crédito Bancário"
responses = {}

original_stdout = sys.stdout
with open(wd_path + '/data/query_engine_output.txt', 'w') as f:
    # Redirect stdout to the file
    sys.stdout = f
    
    for col_name, duvida_map in duvidas.items():
        # if col_name not in not_to_search:
        if col_name in to_search:
            print("\n\n\n------------- " + col_name.upper() + "-------------\n")
            if recursive_search:
                responses[col_name] = recursive_query_engine.query(
                    duvidas[col_name][current_type]
                )
            else:
                responses[col_name] = query_engine_base.query(
                    duvidas[col_name][current_type]
                )
    # Restore the original stdout
    sys.stdout = original_stdout
# 17 seconds

CPU times: user 34.5 ms, sys: 18.4 ms, total: 52.9 ms
Wall time: 1.3 s


In [156]:
for col_name, response in responses.items():
    print("\n\n\n------------- " + col_name.upper() + "-------------\n")
    print(response)

# ------------- DATA DE ASSINATURA-------------

# A data de assinatura, ou data de emissão, é 15/08/2025.
# O documento foi assinado em 18 de novembro de 2020.




------------- DATA DE ASSINATURA-------------

O documento foi assinado em 18 de novembro de 2020.


In [56]:
# ------------- ALTERAÇÃO DE CONTROLE/ REORGANIZAÇÃO-------------

# WITH RECURSIVE: A cláusula de vencimento antecipado menciona que o Banco do Brasil S.A. poderá considerar vencidas antecipadamente as operações de crédito se houver inadimplemento ou se ocorrer a transferência do controle do capital direto e/ou indireto, bem como a substituição de acionistas que resulte em perda do controle societário.

# NO RECURSIVE: A cláusula de vencimento antecipado menciona que o banco pode considerar antecipadamente vencidas as operações de crédito se houver transferência de controle direto e/ou indireto do capital, alteração na titularidade do capital social envolvendo acionistas com variação superior a 10%, ou alteração societária que resulte em perda de controle societário. Também é destacado que qualquer modificação no estatuto ou nos dirigentes deve ser comunicada ao Banco do Brasil durante a vigência da operação de crédito.


### Recursive with reranking

In [71]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,
    node_postprocessors=[reranker],
    verbose=True
)

In [23]:
# !python --version
# Python 3.10.12

response = recursive_query_engine.query(
    # duvidas["Data de Assinatura"][current_type]
    duvidas["Alteração de Controle/ Reorganização"][current_type]
    )
# MIGHT HAVE OUT OF MEMORY ISSUES

In [22]:
print(response)
# A data de assinatura da Cédula de Crédito Bancário é 18 de novembro de 2020 em Ribeirão Preto, São Paulo.

# A Alteração de Controle/Reorganização mencionada na cláusula de vencimento antecipado da cédula de crédito bancário ocorre quando há a transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social com variação superior a 10%, ou mudança na estrutura societária que resulte em perda de controle societário.

Na cláusula de Alteração de Controle/Reorganização, consta que o banco poderá considerar antecipadamente vencidas as obrigações da operação de crédito se houver transferência do controle direto e/ou indireto do capital, alteração de titularidade do capital social envolvendo qualquer acionista com variação superior a 10% do capital social, ou alteração social de qualquer controlada que resulte em perda de controle societário. Além disso, a cláusula prevê a comunicação obrigatória ao Banco do Brasil em caso de modificação do estatuto ou dos dirigentes durante a vigência da operação de crédito.
